In [1]:
from datetime import timedelta
from rich import print
from dynaconf import Dynaconf
from arango import ArangoClient
from utinni import Context
from pathlib import Path

from tiro.plugins.utinni import TiroTSPump
from tiro.plugins.graph.agent import ArangoAgent
from tiro.core import Scenario
from influxdb_client import InfluxDBClient

conf = Dynaconf(settings_files=["./config/utinni_config.toml"])
scenario = Scenario.from_yaml(Path("./config/scenario1.yaml"), Path("./config/use1.yaml"))

In [2]:
context = Context(clients=dict(influxdb=InfluxDBClient(**conf.influxdb),
                               arangodb=ArangoAgent(**conf.arangodb)),
                  **conf.on_request)

context.add_pump("tiro", TiroTSPump(scenario=scenario))

In [3]:
table = context.tiro_table(".*Server%(CPU|Memory)Temperature", column="Server", fill_with_graph=True)

In [4]:
context.bind(start=-timedelta(hours=1),
             step=timedelta(minutes=10))

# table.value
# table["MemoryTemperature"].value
table["CPUTemperature"].value

Server,0ea00ba2-dc1a-11ec-bedc-0242ac130004,0ea00c2e-dc1a-11ec-bedc-0242ac130004,0ea00ca6-dc1a-11ec-bedc-0242ac130004,0ea00d1e-dc1a-11ec-bedc-0242ac130004,0ea00d82-dc1a-11ec-bedc-0242ac130004,0ea00dfa-dc1a-11ec-bedc-0242ac130004,0ea00e68-dc1a-11ec-bedc-0242ac130004,0ea00ed6-dc1a-11ec-bedc-0242ac130004,0ea00f4e-dc1a-11ec-bedc-0242ac130004,0ea00fc6-dc1a-11ec-bedc-0242ac130004,...,0ea04270-dc1a-11ec-bedc-0242ac130004,0ea042d4-dc1a-11ec-bedc-0242ac130004,0ea0432e-dc1a-11ec-bedc-0242ac130004,0ea04392-dc1a-11ec-bedc-0242ac130004,0ea043f6-dc1a-11ec-bedc-0242ac130004,0ea044a0-dc1a-11ec-bedc-0242ac130004,0ea0450e-dc1a-11ec-bedc-0242ac130004,0ea0457c-dc1a-11ec-bedc-0242ac130004,0ea045d6-dc1a-11ec-bedc-0242ac130004,0ea0463a-dc1a-11ec-bedc-0242ac130004
2022-05-25 10:00:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-25 10:10:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-25 10:20:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-25 10:30:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-25 10:40:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-25 10:50:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
2022-05-25 11:00:00+00:00,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0


In [5]:
table = context.tiro_table(Path("./config/use1.yaml"), agg_fn="last")

table.value.keys()

dict_keys(['BackTemperature', 'FrontTemperature', 'MemoryTemperature', 'Temperature', 'CPUTemperature', 'Site'])

In [6]:
context.tiro_table(".*Server%(CPU|Memory)Temperature", type="status", as_dataframe=True, include_tags="Server", value_only=True)["CPUTemperature"].value

,Server,value
0,0ea00ba2-dc1a-11ec-bedc-0242ac130004,20
1,0ea00c2e-dc1a-11ec-bedc-0242ac130004,20
2,0ea00ca6-dc1a-11ec-bedc-0242ac130004,20
3,0ea00d1e-dc1a-11ec-bedc-0242ac130004,20
4,0ea00d82-dc1a-11ec-bedc-0242ac130004,20
...,...,...
118,0ea044a0-dc1a-11ec-bedc-0242ac130004,20
119,0ea0450e-dc1a-11ec-bedc-0242ac130004,20
120,0ea0457c-dc1a-11ec-bedc-0242ac130004,20
121,0ea045d6-dc1a-11ec-bedc-0242ac130004,20


In [8]:
context.tiro_table(Path("./config/pquery.yaml"), type="status", value_only=True, as_dataframe=True, include_tags=["Room", "Rack"]).value

{}

In [25]:
print(context.tiro_table(type="status", value_only=False, as_dataframe=False).mark_dirty().value)

{
    'Room': {
        '0ea0092c-dc1a-11ec-bedc-0242ac130004': {
            'Telemetry': {
                'Temperature': {
                    'timestamp': '2022-05-25T11:10:12.911971',
                    'unit': '°C',
                    'value': 27.62
                }
            },
            'Attribute': {
                'Site': {
                    'timestamp': '2022-05-25T11:02:11.588385',
                    'unit': None,
                    'value': 'Perry-Aguilar'
                }
            },
            'Rack': {
                '0ea00ac6-dc1a-11ec-bedc-0242ac130004': {
                    'Telemetry': {
                        'FrontTemperature': {
                            'timestamp': '2022-05-25T11:10:12.216927',
                            'unit': '°C',
                            'value': 16.48
                        },
                        'BackTemperature': {
                            'timestamp': '2022-05-25T11:10:12.857886',
                            'unit': '°C',
                            'value': 13.42
                        }
                    },
                    'Server': {
                        '0ea00ba2-dc1a-11ec-bedc-0242ac130004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'timestamp': '2022-05-25T11:10:12.905420',
                                    'unit': '°C',
                                    'value': 10.9
                                },
                                'CPUTemperature': {
                                    'timestamp': '2022-05-25T11:02:11.644186',
                                    'unit': '°C',
                                    'value': 20
                                }
                            }
                        },
                        '0ea00c2e-dc1a-11ec-bedc-0242ac130004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'timestamp': '2022-05-25T11:10:12.770307',
                                    'unit': '°C',
                                    'value': 12.3
                                },
                                'CPUTemperature': {
                                    'timestamp': '2022-05-25T11:02:11.708758',
                                    'unit': '°C',
                                    'value': 20
                                }
                            }
                        },
                        '0ea00ca6-dc1a-11ec-bedc-0242ac130004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'timestamp': '2022-05-25T11:10:12.859417',
                                    'unit': '°C',
                                    'value': 19.7
                                },
                                'CPUTemperature': {
                                    'timestamp': '2022-05-25T11:02:11.750922',
                                    'unit': '°C',
                                    'value': 20
                                }
                            }
                        },
                        '0ea00d1e-dc1a-11ec-bedc-0242ac130004': {
                            'Telemetry': {
                                'MemoryTemperature': {
                                    'timestamp': '2022-05-25T11:10:12.311785',
                                    'unit': '°C',
                                    'value': 13.2
                                },
                                'CPUTemperature': {
                                    'timestamp': '2022-05-25T11:02:11.809499',
                                    'unit': '°C',
                                    'value': 20
                                }
                            }
                        },
                        '0ea00d82-dc1a-11ec-bedc-0242ac130004':